In [ ]:
import pandas as pd


In [ ]:
file_one = "Resources/country_vaccinations.csv"
file_one = pd.read_csv(file_one, encoding="ISO-8859-1")

In [ ]:
file_two ="Resources/country_vaccinations_by_manufacturer.csv"
file_two_df= pd.read_csv(file_one, encoding="ISO-8859-1")

In [ ]:
file_one_df.head(5)

In [ ]:
file_two_df.head(5)

In [ ]:
clean_df1=file_one_df.dropna